In [4]:
import pandas as pd


In [10]:
df = pd.read_csv('/content/SMSSpamCollection', sep='\t', header=None, names=['label', 'message'])


In [11]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [12]:
import string

In [14]:
import nltk
from sklearn.model_selection import train_test_split


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords


In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
def clean_text(text):
    text = ''.join([char for char in text if char not in string.punctuation])
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text
df['cleaned'] = df['message'].apply(clean_text)


In [20]:
cv = CountVectorizer()
X = cv.fit_transform(df['cleaned'])

# Encode labels: ham -> 0, spam -> 1
df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})
y = df['label_num']


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

model = MultinomialNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9748878923766816
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       966
           1       0.88      0.95      0.91       149

    accuracy                           0.97      1115
   macro avg       0.93      0.96      0.95      1115
weighted avg       0.98      0.97      0.98      1115



In [26]:
def predict_spam(message):
    cleaned = clean_text(message)
    vectorized = cv.transform([cleaned])
    prediction = model.predict(vectorized)
    return "Spam" if prediction[0] == 1 else "Not Spam"

print(predict_spam("Congratulations! You've won a free ticket!"))
print(predict_spam("Hi teacher, Can we discuss about the final exam?"))


Spam
Not Spam
